<a href="https://colab.research.google.com/github/davidcavazos/predictive-maintenance/blob/master/data-generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predictive Maintenance Data

This notebook will guide you through the process of generating statistically significant random data. This data will then be used to train a BigQuery Machine Learning model to predict a target value.

# Setup

Before you begin, please fill up your Google Cloud project ID and your Cloud Storage bucket name without the `gs://`.

In [0]:
project = "" #@param {type:"string"}
bucket = "" #@param {type:"string"}
dataset = "water_utilities" #@param {type:"string"}
table_training = "pipes_training" #@param {type:"string"}
table = "pipes" #@param {type:"string"}

In [0]:
import numpy as np
import logging
from pprint import pprint

from google.colab import auth
auth.authenticate_user()

def run(command):
  print(f">> {command}")
  !{command}

In [0]:
# Configure bigquery to your project.
run(f"gcloud config set project {project}")
bq = f"bq --project {project}"

In [0]:
# Create a dataset.
run(f"{bq} mk --dataset {dataset}")

# Data Generator

There are different types of fields we can have. For this simple example, we'll only deal with categorical, numerical and dates.

We'll first create a base class `Field` that other fields will inherit from.

In [0]:
class Field:
  def __init__(self, name):
    self.name = name

  def generate_random(self, seed=None):
    raise NotImplementedError('Field.generate_random(seed)')

  def estimate_label(self, value):
    raise NotImplementedError('Field.estimate_label(value)')

The `generate_random()` method will allow us to create a random instance of this field. That is, a random element from the categories, or a random number within a range, or a random date within a range.

The `estimate_label()` method will allow us to *evaluate* a value and get a *score* from `0` to `1`. This value will help us calculate the value we want the model to learn to predict, called *label*. They don't have to be strictly between `0` and `1`, but as long as they are mostly smallish numbers.

We will use the scores from all the fields in a pipe to calculate the *label*, which is the value we want to predict. In this case, our label is the number of years between repairs.

## CategoricalField

For categorical fields, we have a finite number of options.

To estimate the label, we will assume the values have a linear relationship. That is, the first value will be `0`, the next ones will increment by fixed steps until the last one which will be `1`.

You could also make them follow a normal distrbution to get values from `0` to `1`, making different values have different weights.

In [4]:
class CategoricalField(Field):
  def __init__(self, name, categories):
    super().__init__(name)
    self.categories = categories

  def generate_random(self):
    # We only need to make a random choice from all possible categories.
    return np.random.choice(self.categories)

  def estimate_label(self, value):
    # We will assume the last values have more weight than the last,
    # so the first value will give 0 and the last value will give 1.
    try:
      return self.categories.index(value) / (len(self.categories) - 1)
    except Exception as e:
      logging.error(f"{e}: value={value}, valid={self.categories}")
      return 0.5

np.random.seed(42)
field = CategoricalField('Test field name', ['A', 'B', 'C', 'D', 'E'])
print(f"random: {field.generate_random()}")
for value in field.categories:
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:'X' is not in list: value=X, valid=['A', 'B', 'C', 'D', 'E']


random: D
  A label: 0.0
  B label: 0.25
  C label: 0.5
  D label: 0.75
  E label: 1.0
  X label: 0.5


In [5]:
materials = [
    'pvc',
    'ceramic',
    'copper',
    'galvanized iron',
    'cast iron',
]

class Material(CategoricalField):
  def __init__(self):
    super().__init__('material', materials)

  def estimate_label(self, value):
    return 1 - super().estimate_label(value)

np.random.seed(42)
field = Material()
print(f"random: {field.generate_random()}")
for value in field.categories:
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:'X' is not in list: value=X, valid=['pvc', 'ceramic', 'copper', 'galvanized iron', 'cast iron']


random: galvanized iron
  pvc label: 1.0
  ceramic label: 0.75
  copper label: 0.5
  galvanized iron label: 0.25
  cast iron label: 0.0
  X label: 0.5


In [6]:
# Diameter in inches
diameters = [
  1/8, 1/4, 3/8, 1/2, 3/4,
  1, 1 + 1/4, 1 + 1/2,
  2, 2 + 1/2,
  3, 4, 5, 6, 8, 10, 12,
]

class Diameter(CategoricalField):
  def __init__(self):
    super().__init__('diameter_in_inches', diameters)

  def estimate_label(self, value):
    # If the diameter is not one of the predefined categories, we would still
    # like to get a weight that is proportional to where it would have fit
    # within the rest of the numbers.
    if value in self.categories:
      return super().estimate_label(value)
    try:
      return np.searchsorted(self.categories, value) / len(self.categories)
    except Exception as e:
      logging.error(f"{e}: value={value}, valid={self.categories}")
      return 0.5


np.random.seed(42)
field = Diameter()
print(f"random: {field.generate_random()}")
for value in field.categories:
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  9 label: {field.estimate_label(9)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe': value=X, valid=[0.125, 0.25, 0.375, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 6, 8, 10, 12]


random: 1.25
  0.125 label: 0.0
  0.25 label: 0.0625
  0.375 label: 0.125
  0.5 label: 0.1875
  0.75 label: 0.25
  1 label: 0.3125
  1.25 label: 0.375
  1.5 label: 0.4375
  2 label: 0.5
  2.5 label: 0.5625
  3 label: 0.625
  4 label: 0.6875
  5 label: 0.75
  6 label: 0.8125
  8 label: 0.875
  10 label: 0.9375
  12 label: 1.0
  9 label: 0.8823529411764706
  X label: 0.5


## NumericalField

For numbers, we'll also just use a linear uniform distribution, but you could also use any kind of distribution or function that best fits your data.

In [7]:
class NumericalField(Field):
  def __init__(self, name, min_value, max_value):
    super().__init__(name)
    self.min_value = min_value
    self.max_value = max_value

  def generate_random(self):
    # We'll just get a uniform distribution between two values.
    return np.random.uniform(self.min_value, self.max_value)

  def estimate_label(self, value):
    # The first value will be 0 and the last will be 1.
    try:
      return (value - self.min_value) / (self.max_value - self.min_value)
    except Exception as e:
      logging.error(f"{e}: value={value}, range=({self.min_value}, {self.max_value})")
      return 0.5

np.random.seed(42)
field = NumericalField('Test field name', 5, 10)
print(f"random: {field.generate_random()}")
for value in np.linspace(field.min_value, field.max_value, 10):
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  0.0 label: {field.estimate_label(0.0)}")
print(f"  15.0 label: {field.estimate_label(15.0)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:unsupported operand type(s) for -: 'str' and 'int': value=X, range=(5, 10)


random: 6.872700594236813
  5.0 label: 0.0
  5.555555555555555 label: 0.11111111111111108
  6.111111111111111 label: 0.22222222222222215
  6.666666666666667 label: 0.33333333333333337
  7.222222222222222 label: 0.4444444444444445
  7.777777777777778 label: 0.5555555555555556
  8.333333333333334 label: 0.6666666666666667
  8.88888888888889 label: 0.7777777777777779
  9.444444444444445 label: 0.888888888888889
  10.0 label: 1.0
  0.0 label: -1.0
  15.0 label: 2.0
  X label: 0.5


In [8]:
# Length in miles
length_min = 0.1
length_max = 10

class Length(NumericalField):
  def __init__(self):
    super().__init__('length_in_miles', length_min, length_max)

np.random.seed(42)
field = Length()
print(f"random: {field.generate_random()}")
for value in np.linspace(field.min_value, field.max_value, 10):
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  15.0 label: {field.estimate_label(15.0)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:unsupported operand type(s) for -: 'str' and 'float': value=X, range=(0.1, 10)


random: 3.807947176588889
  0.1 label: 0.0
  1.2000000000000002 label: 0.11111111111111112
  2.3000000000000003 label: 0.22222222222222224
  3.4000000000000004 label: 0.33333333333333337
  4.5 label: 0.4444444444444445
  5.6 label: 0.5555555555555556
  6.7 label: 0.6666666666666667
  7.800000000000001 label: 0.7777777777777779
  8.9 label: 0.888888888888889
  10.0 label: 1.0
  15.0 label: 1.505050505050505
  X label: 0.5


## DateField

Dates are a bit more complicated since they have years, months, days, hours, minutes, seconds and milliseconds. And not even taking into account timezones and leap seconds.

Fortunately there is a simple solution. We can transform them to *Unix time*, which is basically a number representing the number of seconds that have passed since January 1, 1970. The standard library already has functions that take care of this conversion for us, taking leap seconds and everything into account.

Transformed as a number, we can now use the same method as with numbers going between 0 and the current time.

In [9]:
from datetime import datetime

class DateField(NumericalField):
  def __init__(self, name):
    super().__init__(name, 0, datetime.now().timestamp())

np.random.seed(42)
field = DateField('Test field name')
print(f"random: {field.generate_random()}")
for value in np.linspace(field.min_value, field.max_value, 10):
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:unsupported operand type(s) for -: 'str' and 'int': value=X, range=(0, 1554441094.707533)


random: 582200552.3529836
  0.0 label: 0.0
  172715677.18972588 label: 0.1111111111111111
  345431354.37945175 label: 0.2222222222222222
  518147031.5691776 label: 0.3333333333333333
  690862708.7589035 label: 0.4444444444444444
  863578385.9486294 label: 0.5555555555555556
  1036294063.1383553 label: 0.6666666666666666
  1209009740.3280811 label: 0.7777777777777778
  1381725417.517807 label: 0.8888888888888888
  1554441094.707533 label: 1.0
  X label: 0.5


In [10]:
class InstallationDate(DateField):
  def __init__(self):
    super().__init__('installation_date')

np.random.seed(42)
field = InstallationDate()
print(f"random: {field.generate_random()}")
for value in np.linspace(field.min_value, field.max_value, 10):
  print(f"  {value} label: {field.estimate_label(value)}")
print(f"  X label: {field.estimate_label('X')}")

ERROR:root:unsupported operand type(s) for -: 'str' and 'int': value=X, range=(0, 1554441095.319178)


random: 582200552.5820693
  0.0 label: 0.0
  172715677.25768647 label: 0.11111111111111112
  345431354.51537293 label: 0.22222222222222224
  518147031.77305937 label: 0.3333333333333333
  690862709.0307459 label: 0.4444444444444445
  863578386.2884324 label: 0.5555555555555556
  1036294063.5461187 label: 0.6666666666666666
  1209009740.8038054 label: 0.7777777777777779
  1381725418.0614917 label: 0.888888888888889
  1554441095.319178 label: 1.0
  X label: 0.5


## Schema

We'll now put together all the fields into a schema. This will allow us to access the columns and to generate new pipes.

Note that we also have a `columns_training()` method, which basically adds a label column. This `label` column is the value the Machine Learning model will try to learn to predict.

In this case we'll name the number of years between repairs as the **`lifespan`**.

In [11]:
import re

class Schema:
  def __init__(self, fields, id_name='id', label_name='label'):
    self.fields = fields
    self.id_name = id_name
    self.label_name = label_name

  def columns(self):
    return [self.id_name] + [field.name for field in self.fields]

  def columns_training(self):
    return self.columns() + [self.label_name]

  def generate_random(self, element_id, seed=None):
    np.random.seed(seed)
    pipe = {field.name: field.generate_random() for field in self.fields}
    pipe[self.id_name] = element_id
    return pipe

fields = [
    Material(),
    InstallationDate(),
    Diameter(),
    Length(),
]
schema = Schema(fields, id_name='pipe_id', label_name='lifespan')

print(schema.columns())
print(schema.columns_training())
schema.generate_random(0, seed=42)

['pipe_id', 'material', 'installation_date', 'diameter_in_inches', 'length_in_miles']
['pipe_id', 'material', 'installation_date', 'diameter_in_inches', 'length_in_miles', 'lifespan']


{'diameter_in_inches': 3.0,
 'installation_date': 1477829388.597788,
 'length_in_miles': 7.818940902700415,
 'material': 'galvanized iron',
 'pipe_id': 0}

# Training the model

We're almost ready to generate the data, but we're still missing the data labels. We  need to calculate the value we want the model to learn to predict. And we'll use the `estimate_label()` methods we created earlier.

## Data labeling

First, we'll calculate a *weight* from `0` to `1` using the `estimate_label()` methods. We can then scale this into any other range. Eventually, we'll scale it to values from `0` to `100`, since it's roughly a good estimate of the lifespan of pipes.

Remember, the weights don't have to be strictly between `0` and `1`, because in the end the model might predict `-0.1` or `1.2`, or even an occasional `3.14`, but most values will be between `0` and `1`. So predictions can fall outside our predefined range of `0` to `100` years.

In [0]:
pipe = schema.generate_random(0, seed=42)

lifespan_weight = 0.0
print("field: value; weight * lifespan = weighted_lifespan")
for weight, field in zip(weights, schema.fields):
  label = field.estimate_label(pipe[field.name])
  weighted_label = weight * label

  print(f"  {field.name}: {weight} * {label} = {weighted_label}")
  lifespan_weight += weighted_label
print(f"lifespan_weight: {lifespan_weight}")

field: value; weight * lifespan = weighted_lifespan
  material: 0.4 * 0.25 = 0.1
  installation_date: 0.3 * 0.9507143064099162 = 0.2852142919229748
  diameter_in_inches: 0.2 * 0.625 = 0.125
  length_in_miles: 0.1 * 0.7796910002727693 = 0.07796910002727693
lifespan_weight: 0.5881833919502518


In [0]:
def calculate_lifespan_years(schema, pipe):
  # We'll assume the fields are sorted by importance.
  # That is, the first field will have the largest weight to lifespan, and the
  # last field will have the least weight.
  indices = np.array(range(len(schema.fields), 0, -1))
  weights = indices / indices.sum()

  # Calculate a weighted sum for the lifespan estimate of each field.
  lifespan_weight = 0.0
  for weight, field in zip(weights, schema.fields):
    lifespan_weight += weight * field.estimate_label(pipe[field.name])

  # Return in a scale from 0 to 100 years.
  return 100.0 * lifespan_weight

pipe = schema.generate_random(0, seed=42)
pprint(pipe)

lifespan_years = calculate_lifespan_years(schema, pipe)
print(f"lifespan_years: {lifespan_years}")

{'diameter_in_inches': 3.0,
 'installation_date': 1477809099.538208,
 'length_in_miles': 7.818940902700415,
 'material': 'galvanized iron',
 'pipe_id': 0}
lifespan_years: 58.81833919502518


## Generating the training data

We're finally ready to generate our training data!

We'll just write it to a CSV file since we can easily load that into BigQuery.

In [0]:
training_points = 1000000 #@param {type:"integer"}

In [0]:
import csv

np.random.seed(42)

training_data_file = 'data.training.csv'
with open(training_data_file, 'w') as f:
  writer = csv.DictWriter(f, schema.columns_training())
  writer.writerow({col: col for col in schema.columns_training()})

  for i in range(training_points):
    # Get all the fields in the schema plus the lifespan (the model label).
    pipe = schema.generate_random(f"pipe {i}")
    pipe[schema.label_name] = calculate_lifespan_years(schema, pipe)
    writer.writerow(pipe)

!ls -lh {training_data_file}
!head {training_data_file}

-rw-r--r-- 1 root root 78M Apr  5 00:08 data.training.csv
pipe_id,material,installation_date,diameter_in_inches,length_in_miles,lifespan
pipe 0,cast iron,733799626.8085197,4.0,1.1394253942982249,28.96211521315033
pipe 1,copper,466256102.76490426,3.0,5.052582032218517,46.50126052701041
pipe 2,cast iron,388383361.23934716,10.0,3.842743531008332,30.02627261672147
pipe 3,pvc,982745238.7999034,0.25,2.773272528118681,62.91706791419678
pipe 4,cast iron,538849234.2931997,6.0,4.886764052620875,31.484800986829608
pipe 5,pvc,408205690.55501634,1.0,6.534157697395788,60.62743989862044
pipe 6,copper,700831520.0825703,1.25,7.060759651579949,48.05698285764014
pipe 7,pvc,1396897870.094761,1.25,7.998648315122436,82.43829078778198
pipe 8,ceramic,993421118.277087,5.0,0.9596218247364392,65.04113991145189


## Uploading data to BigQuery

First, we'll copy it to Cloud Storage so BigQuery can load it.

In [0]:
gcs_training_data_file = f"gs://{bucket}/{training_data_file}"
run(f"gsutil cp {training_data_file} {gcs_training_data_file}")

>> gsutil cp data.training.csv gs://pdm-2018/data.training.csv
Copying file://data.training.csv [Content-Type=text/csv]...
\
Operation completed over 1 objects/77.5 MiB.                                     


In [0]:
# Load the data into the table.
run(f"{bq} load --source_format=CSV --autodetect --replace "
    f"{dataset}.{table_training} {gcs_training_data_file}")

>> bq --project pdm-2018 load --source_format=CSV --autodetect --replace water_utilities.pipes_training gs://pdm-2018/data.training.csv
Waiting on bqjob_r2646c0881d246eea_00000169ead4a083_1 ... (15s) Current status: DONE   


## Training a BigQuery ML Model

First, let's get 3 rows to see how our data looks like.

In [0]:
%%bigquery --project {project}
SELECT * FROM `water_utilities.pipes_training` LIMIT 3

,pipe_id,material,installation_date,diameter_in_inches,length_in_miles,lifespan
0,pipe 62,pvc,1.100928e+09,2.0,0.158857,71.307151
1,pipe 76,pvc,1.609211e+08,2.0,5.175855,58.232873
2,pipe 100,pvc,1.511691e+09,2.0,0.216494,79.293020


Next, we can create and train a BigQuery ML model with a very simple SQL [CREATE MODEL statement](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create). It is very similar to creating a table, and we'll be able to access it as if it were a table. The only difference is that rather than having the values *stored* in the database, they will be computed as needed.

We'll use linear regression to train our model since we're predicting a number. If you're predicting a classification, you would want to use logistic regression (`logistic_reg`) instead.

In [0]:
%%bigquery --project {project}
CREATE OR REPLACE MODEL
  `water_utilities.pipe_lifespan`
OPTIONS (
  model_type='linear_reg',
  input_label_cols=['lifespan']
)
AS SELECT * FROM `water_utilities.pipes_training`

""


## Evaluating the model

So, something happened, but we don't really know any details of it. How can we measure if the model is doing good or bad?

Fortunately, BigQuery ML provides us with an [ML.EVALUATE function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate) which will give us more details on the error and variance of our model.

In [0]:
%%bigquery --project {project}
SELECT *
FROM
  ML.EVALUATE(
    MODEL `water_utilities.pipe_lifespan`,
    (SELECT * FROM `water_utilities.pipes_training`)
  )

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,4.168218,24.778829,0.025295,3.979697,0.922795,0.968233


## Getting predictions

We can get prediction results with a simple SELECT statement using the [ML.PREDICT function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict), and access it as if it were another table.

In [0]:
%%bigquery --project {project}
SELECT
  predicted_lifespan
FROM
  ML.PREDICT(
    MODEL `water_utilities.pipe_lifespan`,
    (SELECT * FROM `water_utilities.pipes_training` LIMIT 3)
  )

,predicted_lifespan
0,71.917459
1,60.427110
2,78.944235


Let's compare some of our generated labels with the predicted values. They are surprisingly similar, so it's looking good!

In [0]:
%%bigquery --project {project}
SELECT
  lifespan,
  predicted_lifespan
FROM
  ML.PREDICT(
    MODEL `water_utilities.pipe_lifespan`,
    (SELECT * FROM `water_utilities.pipes_training` LIMIT 10)
  )

,lifespan,predicted_lifespan
0,71.307151,71.917459
1,58.232873,60.427110
2,79.293020,78.944235
3,75.898893,75.978790
4,72.430876,72.922252
5,69.135648,70.014202
6,69.630158,70.466734
7,79.228437,78.892358
8,56.831197,59.188136
9,71.027719,71.674389


# Generating the app data

We'll now generate a second dataset. This will be to populate the app since we don't have real data, but you could skip this step if you already have data.

For this dataset, however, we no longer need the label. And we'll also add `last_repair_date` and `repair_cost` columns. We'll add the predicted lifespan to the last repair date to get the predicted break date.

We'll also create a simple repair cost estimate to test the budget analysis on the app.

In [0]:
total_pipes = 1000 #@param {type:"integer"}

In [21]:
import csv

np.random.seed(42)

data_file = 'data.csv'
with open(data_file, 'w') as f:
  columns = schema.columns() + ['last_repair_date', 'repair_cost']
  writer = csv.DictWriter(f, columns)
  writer.writerow({col: col for col in columns})

  for i in range(total_pipes):
    pipe = schema.generate_random(f"pipe {i}", seed=i)

    # The last repair date is any date between the installation date and now.
    pipe['last_repair_date'] = np.random.uniform(pipe['installation_date'], datetime.now().timestamp())

    # The repair cost will be around $10,000 and affected by the pipe length.
    pipe['repair_cost'] = np.log(pipe['length_in_miles']) * 10000

    writer.writerow(pipe)

!ls -lh {data_file}
!head {data_file}

-rw-r--r-- 1 root root 95K Apr  5 05:16 data.csv
pipe_id,material,installation_date,diameter_in_inches,length_in_miles,last_repair_date,repair_cost
pipe 0,cast iron,921542038.2105968,0.125,6.067357423109274,1266398254.849563,18029.231598850518
pipe 1,galvanized iron,1119701995.1824014,2.5,3.0930924690552137,1183502563.236167,11291.71389491165
pipe 2,pvc,287699188.47648627,2.0,5.541658530999221,839140437.1262035,17122.938297823643
pipe 3,copper,109937660.01058717,0.5,2.979956915238149,847830044.2121457,10919.0884243902
pipe 4,copper,1399963002.5420372,1.0,9.729575163652354,1510386628.9287405,22751.702327217958
pipe 5,galvanized iron,85774252.39892076,1.25,3.7009952812958358,1524252941.7219849,13086.017784223459
pipe 6,copper,1472795751.060424,3.0,0.7361205854475242,1503013571.9554102,-3063.613347501248
pipe 7,cast iron,1212337822.33122,0.5,9.784406681077966,1368195049.3324254,22807.89963452958
pipe 8,galvanized iron,17276743.26681634,2.5,8.705025948117807,833289348.0773457,21639.0055404

Next, we'll also copy it to Cloud Storage and upload it into BigQuery to another table.

In [22]:
gcs_data_file = f"gs://{bucket}/{data_file}"
run(f"gsutil cp {data_file} {gcs_data_file}")

>> gsutil cp data.csv gs://pdm-2018/data.csv
Copying file://data.csv [Content-Type=text/csv]...
/ [1 files][ 94.8 KiB/ 94.8 KiB]                                                
Operation completed over 1 objects/94.8 KiB.                                     


In [26]:
# Load the data into the table.
run(f"{bq} load --source_format=CSV --autodetect --replace "
    f"{dataset}.{table} {gcs_data_file}")

>> bq --project pdm-2018 load --source_format=CSV --autodetect --replace water_utilities.pipes gs://pdm-2018/data.csv
Waiting on bqjob_rc86524071141114_00000169ebee053a_1 ... (1s) Current status: DONE   


In [0]:
%%bigquery --project {project}
SELECT * FROM `water_utilities.pipes` LIMIT 3

,pipe_id,material,installation_date,diameter_in_inches,length_in_miles,last_repair_date
0,pipe 2,pvc,2.876952e+08,2.0,5.541659,8.391293e+08
1,pipe 57,pvc,4.830868e+08,2.0,8.081632,1.487070e+09
2,pipe 75,pvc,1.107097e+09,2.0,3.333048,1.266758e+09


Finally, we can get predictions on our newly created dataset.

In [27]:
%%bigquery --project {project}
SELECT
  pipe_id,
  material,
  diameter_in_inches,
  length_in_miles,
  installation_date,
  last_repair_date,
  last_repair_date + predicted_lifespan*60*60*24*365 AS predicted_break_date
FROM
  ML.PREDICT(
    MODEL `water_utilities.pipe_lifespan`,
    (SELECT * FROM `water_utilities.pipes` LIMIT 3)
  )

,pipe_id,material,diameter_in_inches,length_in_miles,installation_date,last_repair_date,predicted_break_date
0,pipe 2,pvc,2.0,5.541659,2.876992e+08,8.391404e+08,2.382652e+09
1,pipe 57,pvc,2.0,8.081632,4.830934e+08,1.487089e+09,3.280129e+09
2,pipe 75,pvc,2.0,3.333048,1.107112e+09,1.266775e+09,3.435509e+09
